In [1]:
%cd ~/bioinfo1/project/binfo1-work/

/rna/hyemin/bioinfo1/project/binfo1-work


##### Base positions with cut-offs of 0.8 for crosslinking-induced reverse-transcription error score (CRES) and 50 for read depth.

In [8]:
!samtools mpileup --ff 3844 CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup # filter out QC failed, unmapped, secondary, supplementary, and duplicate reads

[mpileup] 1 samples in 1 input files
959691175 CLIP-35L33G.pileup


In [2]:
!awk '$4 >= 50 {print $0}' CLIP-35L33G.pileup > CLIP-35L33G-ge50.pileup

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [4]:
dfPileup = pd.read_csv('CLIP-35L33G-ge50.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
dfPileup.tail()

,chrom,pos,_ref,count,basereads,quals
25209663,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...
25209664,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...
25209665,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...
25209666,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...
25209667,MU069435.1,1568,N,88,>>>>T$T$T$T$T$T$T$T$T$T$TT$T$TT$T$T$T$T$T$T$T$...,DIDHAG4BDBHBEIGHIHHHIIHIDIIIIIHIGIHIIEIHIHDHIG...


In [6]:
toremove = re.compile('[<>$]|\^.') # [문자들] : 문자들 중 하나와 매치
dfPileup['matchesNdel'] = dfPileup['basereads'].apply(lambda x: toremove.sub('', x)) # remove special characters (<>$^.) from basereads
dfPileup['AGCTD'] = dfPileup['matchesNdel'].apply(lambda x: np.array((x.count('A') + x.count('a'), \
                                                                      x.count('G') + x.count('g'), \
                                                                      x.count('C') + x.count('c'), \
                                                                      x.count('T') + x.count('t'), \
                                                                      x.count('*') + x.count('#'))))
dfPileup['entropy'] = dfPileup['AGCTD'].apply(lambda x: -sum([i*np.log2(i) for i in x/sum(x) if i > 0]))
dfPileup['entropy'] = dfPileup['entropy'].replace(-0.0, 0.0)
dfPileup[dfPileup['entropy'] >= 0.8].head()

,chrom,pos,_ref,count,basereads,quals,matchesNdel,AGCTD,entropy
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,"[55, 81, 20, 0, 0]",1.401164
46,chr1,3224576,N,168,CCCG*GGGG*GGGGC*GGGGGGGGGG**GGGGGCTCTGCCCCCCCG...,BGI?H?G8HIEGHIIIBIHFIE9GIIG?=E?:HHGBA8GCB??0GH...,CCCG*GGGG*GGGGC*GGGGGGGGGG**GGGGGCTCTGCCCCCCCG...,"[0, 93, 53, 12, 10]",1.511601
136,chr1,3872651,N,50,ccccccccc*cccgcgc*cccccccccc*ccccc*gc*g*c-1n**...,EGIIEIHIFGGGIGBGGHGIIIIG@HHIHII<EGHEHIGH9HHHDG...,ccccccccc*cccgcgc*cccccccccc*ccccc*gc*g*c-1n**...,"[0, 4, 33, 0, 13]",1.192442
175,chr1,4329961,N,55,<<cccccccccccccccccccccccccctcctccct*tttttt*tt...,8IDGDH6HHGIGD<EHGBIGGDHBIIG>HH8HEIIIHGIFHIGHGH...,cccccccccccccccccccccccccctcctccct*tttttt*ttcc...,"[0, 0, 40, 11, 2]",0.955640
176,chr1,4329962,N,55,<<ccccccccccccc$tttc$c$c$ttttc$ttcggcgggcccccc...,8IDGDG=HH@IGBBEHGIIHEEHGFIGDIH?HDHIIHGGIHIHHGG...,ccccccccccccctttcccttttcttcggcgggccccccccccctt...,"[0, 5, 31, 17, 0]",1.300060


In [8]:
dfPileup[dfPileup['entropy'] >= 0.8].to_csv('CLIP-35L33G-ge50-entropy.mpileup', sep='\t', index=False, header=False)

##### Neighboring sequences of the high-mutated bases in reference genome (mm39).

In [10]:
!wget -P ~/bioinfo1/project/binfo1-datapack1 https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M35/GRCm39.primary_assembly.genome.fa.gz
!gunzip ~/bioinfo1/project/binfo1-datapack1/GRCm39.primary_assembly.genome.fa.gz

--2024-05-24 23:48:06--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M35/GRCm39.primary_assembly.genome.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773873008 (738M) [application/x-gzip]
Saving to: ‘/rna/hyemin/bioinfo1/project/binfo1-datapack1/GRCm39.primary_assembly.genome.fa.gz’

GRCm39.primary_asse 100%[===================>] 738.02M   194KB/s    in 60m 44s 

2024-05-25 00:48:51 (207 KB/s) - ‘/rna/hyemin/bioinfo1/project/binfo1-datapack1/GRCm39.primary_assembly.genome.fa.gz’ saved [773873008/773873008]



In [11]:
!rsync -avz ~/bioinfo1/project/binfo1-datapack1/GRCm39.primary_assembly.genome.fa ./

sending incremental file list
GRCm39.primary_assembly.genome.fa

sent 830,138,257 bytes  received 35 bytes  40,494,550.83 bytes/sec
total size is 2,773,693,944  speedup is 3.34


In [12]:
dfPileupFiltered = pd.read_csv('CLIP-35L33G-ge50-entropy.mpileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'matchesNdel', 'AGCTD', 'entropy'])

In [13]:
dfNeighborBed15 = dfPileupFiltered[['chrom', 'pos']].copy()
dfNeighborBed15['start'] = dfNeighborBed15['pos'] - 1
dfNeighborBed15['neiStart'] = dfNeighborBed15['start'] - 15
dfNeighborBed15['neiEnd'] = dfNeighborBed15['start'] + 15
dfNeighborBed15[['chrom', 'neiStart', 'neiEnd']].to_csv('CLIP-35L33G-ge50-entropy-neighbor15.bed', sep='\t', index=False, header=False)

In [15]:
!bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed CLIP-35L33G-ge50-entropy-neighbor15.bed -bedOut > CLIP-35L33G-ge50-entropy-neighbor15-seq.bed